In [1]:
import os
import json

from itertools import groupby
from config import iam_policy_data
import preprocessiong_iam_policy

def group_permissions_by_service(policy_lists):
    grouped_permissions = {}
    for policy_list in policy_lists:
        if policy_list["Service Effect"] != iam_policy_data.service_effect[0]:
            service = policy_list["Service"]
            if service not in grouped_permissions:
                grouped_permissions[service] = {"Action": [], "Resource": "*"}
            grouped_permissions[service]["Action"].append(policy_list["Actions"])
        else:
            pass
    return grouped_permissions

def generate_iam_policy(grouped_permissions):
    iam_policy = {
        "Version": "2012-10-17",
        "Statement": [],
    }

    for service, actions_resources in grouped_permissions.items():
        statement = {
            "Sid": f"{service.lower()}allowfordevops",
            "Effect": "Allow",
            "Action": actions_resources["Action"],
            "Resource": actions_resources["Resource"],
        }
        iam_policy["Statement"].append(statement)

    return iam_policy

def save_iam_policy_to_file(iam_policy, category, output_directory):
    output_file_path = os.path.join(output_directory, f"{category.lower()}_iam_policy_devops")
    with open(output_file_path, 'w') as json_file:
        json.dump(iam_policy, json_file, indent=4)

# def main():
#     policy_lists = preprocessiong_iam_policy.excel_to_json(iam_policy_data.input_file_path, iam_policy_data.sheet_name)
#     grouped_by_category = {key: list(group) for key, group in groupby(policy_lists, key=lambda x: x['Category'])}
    
#     for category, items in grouped_by_category.items():
#         print(f"Category: {category}")
#         grouped_permissions = group_permissions_by_service(items)
#         iam_policy = generate_iam_policy(grouped_permissions)    
#         save_iam_policy_to_file(iam_policy, category , iam_policy_data.output_directory)

# if __name__ == "__main__":
#     main()


In [2]:
policy_lists = preprocessiong_iam_policy.excel_to_json(iam_policy_data.input_file_path, iam_policy_data.sheet_name)

In [3]:
# Category와 Wildcard로 데이터 정렬
sorted_data = sorted(policy_lists, key=lambda x: (x['Category'], x['Wildcard']))

# Category와 Wildcard로 그룹화
grouped_by_category_and_wildcard = {key: list(group) for key, group in groupby(sorted_data, key=lambda x: (x['Category'], x['Wildcard']))}

# Grouped 결과를 저장할 리스트
grouped_actions = []
personal_actions = []

# 그룹화된 데이터에서 Service Effect가 모두 같은 경우 Action을 하나로 처리
for key, group in grouped_by_category_and_wildcard.items():
    category, wildcard = key
    service_effects = set(item['Service Effect'] for item in group)
    if len(service_effects) == 1:
        # Service Effect가 모두 같으면 하나로 처리
        action = f"{group[0]['Actions'].split(':')[0].lower()}:{wildcard}"
        grouped_actions.append({'Wildcard' : wildcard, 'Action': action, 'Category': category, 'Service': group[0]['Service'], 'Service Effect': service_effects.pop()})

# # Action 값의 ":" 뒤의 값이 policy_lists의 Wildcard 값과 다른 경우 추가
# for item in policy_lists:
#     action_name = item['Actions'].split(':')[1]
#     if any(action_name not in grouped_item['Action'] for grouped_item in grouped_actions):
#         grouped_actions.append(item)


In [4]:
grouped_actions

[{'Wildcard': 'Allocate*',
  'Action': 'ec2:Allocate*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Assign*',
  'Action': 'ec2:Assign*',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Attach*',
  'Action': 'ec2:Attach*',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Authorize*',
  'Action': 'ec2:Authorize*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Cancel*',
  'Action': 'ec2:Cancel*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Confirm*',
  'Action': 'ec2:Confirm*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Copy*',
  'Action': 'ec2:Copy*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Delete*',
  'Action': 'ec2:Delete*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effe

In [5]:
#### grouped_actions_dict를 'Wildcard'를 키로 하는 딕셔너리로 변경
grouped_actions_dict_by_wildcard = {item['Wildcard']: item for item in grouped_actions}

# 삭제할 인덱스를 저장할 리스트 초기화
indices_to_remove = []

# policy_lists 순회
for idx, policy in enumerate(policy_lists):
    wildcard = policy.get('Wildcard')
    category = policy.get('Category')
    service = policy.get('Service')
    print(category)

    # wildcard이 grouped_actions_dict에 존재하면서 값이 일치하는 경우
    try:
        if grouped_actions_dict_by_wildcard[wildcard]['Wildcard'] == wildcard and grouped_actions_dict_by_wildcard[wildcard]['Category'] == category and grouped_actions_dict_by_wildcard[wildcard]['Service'] == service:
            indices_to_remove.append(idx)
            print(idx)
    except KeyError:
        # KeyError가 발생하면 예외 처리
        print(f"KeyError: {wildcard} not found in grouped_actions_dict_by_wildcard")

# 저장된 인덱스를 역순으로 정렬하여 뒤에서부터 삭제
for idx in reversed(indices_to_remove):
    del policy_lists[idx]


Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Security
3
Security
4
Security
5
Storage
Storage
Storage
Storage
Security
10
Security
11
Storage
Storage
Storage
Storage
Infra
16
Infra
17
Storage
18
Storage
19
Infra
20
Infra
21
Infra
22
Infra
KeyError: Assosiate* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Assosiate* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Assosiate* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Assosiate* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Assosiate* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Assosiate* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Assosiate* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Assosiate* not found in grouped_actions_dict_by_wildcard
Security
31
Securi

In [4]:
grouped_actions_dict_by_wildcard = {item['Wildcard']: item for item in grouped_actions}

In [16]:
# grouped_actions_dict_by_wildcard
output_file_path = os.path.join('./', "group_test.json")
with open(output_file_path, 'w') as json_file:
    json.dump(grouped_actions_dict_by_wildcard, json_file, indent=4)

In [14]:
policy_lists[200:250]

[{'Wildcard': 'Describe*',
  'Actions': 'ec2:DescribeBundleTasks',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Describe*',
  'Actions': 'ec2:DescribeByoipCidrs',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Describe*',
  'Actions': 'ec2:DescribeCapacityReservationFleets',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Describe*',
  'Actions': 'ec2:DescribeCapacityReservations',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Describe*',
  'Actions': 'ec2:DescribeElasticGpus',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Describe*',
  'Actions': 'ec2:DescribeFastLaunchImages',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Describe*',
  'Actions': 'ec2:DescribeFleetHistory',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effec

In [11]:
grouped_actions

[{'Wildcard': 'Allocate*',
  'Action': 'ec2:Allocate*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Assign*',
  'Action': 'ec2:Assign*',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Attach*',
  'Action': 'ec2:Attach*',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Authorize*',
  'Action': 'ec2:Authorize*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Cancel*',
  'Action': 'ec2:Cancel*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Confirm*',
  'Action': 'ec2:Confirm*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Copy*',
  'Action': 'ec2:Copy*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Delete*',
  'Action': 'ec2:Delete*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effe

In [8]:
print(len(policy_lists))
print(len(grouped_actions))

445
67


In [10]:
441 + 67

508

In [9]:
policy_lists.extend(grouped_actions)

In [10]:
len(policy_lists)

512

In [13]:
group_dict = preprocessiong_iam_policy.group_dict_by_key(policy_lists, 'Category')

In [8]:
for category, items in group_dict.items():
    print(f"{category} IAM Policy created for devops")
    grouped_permissions = group_permissions_by_service(items)
    iam_policy = generate_iam_policy(grouped_permissions)    
    save_iam_policy_to_file(iam_policy, category, iam_policy_data.output_directory)


[{'Wildcard': 'Accept*',
  'Actions': 'ec2:AcceptAddressTransfer',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Accept*',
  'Actions': 'ec2:AcceptReservedInstancesExchangeQuote',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Accept*',
  'Actions': 'ec2:AcceptVpcPeeringConnection',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Add*',
  'Actions': 'rds:AddRoleToDBCluster',
  'Category': 'Storage',
  'Service': 'RDS',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Add*',
  'Actions': 'rds:AddRoleToDBInstance',
  'Category': 'Storage',
  'Service': 'RDS',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Add*',
  'Actions': 'rds:AddSourceIdentifierToSubscription',
  'Category': 'Storage',
  'Service': 'RDS',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Add*',
  'Actions': 'rds:AddTagsToResource',
  'Category': 'Storage',
  'Service': 'RDS',
  'Service Effect': '중 (중